In [59]:
import pandas as pd
import numpy as np
import pickle
import warnings
import matplotlib.pyplot as plt

## Estimating attractiveness by layers

Likelihood function:
$$
L(a) = \prod^n_1 a_i^{k_i}
$$
Log likelihood function:
$$
l(a) = \sum^n_1k_i\ln{a_i}
$$
Where $a$ represents the probability of selecting the container given that the parent container has been selected. $k$ represents the number of times the container has been selected.

#### Estimating the attractiveness of all sub-containers within a given timeframe
- flow_mat: Matrix $M$ representing the flow of personnel between institutions during a certain period of time, where $m_{i, j}$ denotes the number of individuals flowing from institution i to institution j within that timeframe.
- h: The level of container to be estimated.
    - Example: To estimate the attractiveness of 10 cities in China, where cities are at level 0, h should be set to $h = 0$.
- ins_geo: Geographic information of all institutions within the specified container.
    - Example: To consider the attractiveness of 10 cities in China, the geographic information table of all institutions within these 10 cities should be provided.
- all_geo: Geographic information of all institutions in the top 30 countries.


In [60]:
def get_level_distance(ins_1, ins_2, all_geo):
    row_1 = all_geo.loc[ins_1]
    row_2 = all_geo.loc[ins_2]
    # Considering many countries have cities with the same name, level distance should be judged from top to bottom.
    if(row_1['continent_id'] != row_2['continent_id']):
        return 3 # Intercontinental movement
    elif(row_1['country_id'] != row_2['country_id']):
        return 2 # Movement between different countries within the same continent
    elif(row_1['city_id'] != row_2['city_id']):
        return 1 # Movement between different cities within the same country
    else: return 0 # Movement within the same city
    
    
def get_n(h, ins_geo): # Find the number of containers at the specified level
    if(h == 0): return len(ins_geo[['city_id']].drop_duplicates())
    elif(h == 1): return len(ins_geo[['country_id']].drop_duplicates())
    elif(h == 2): return len(ins_geo[['continent_id']].drop_duplicates())

def get_k(ins_geo, all_geo, flow_mat, h, n):
    k = np.zeros(n)
    for i in all_geo.index:
        for j in ins_geo.index:
            flow = flow_mat[i, j]
            if(flow != 0):
                d = get_level_distance(i, j, all_geo)
                if (d >= h+1):
                    id = ins_geo.loc[j][h] # Get the geographic location of institution j at level h
                    k[id] += flow
    return k


In [61]:
def estimate_by_slice(flow_mat, h, ins_geo, all_geo):
    n = get_n(h, ins_geo)
    k = get_k(ins_geo, all_geo, flow_mat, h, n)
    return [k[i]/sum(k) for i in range(n)]

In [62]:
def estimate_this_country(code:str, all_geo, ins_geo):
    '''
    Input: Country code, geographic data of all institutions in the top 30 countries, geographic data of institutions in the top 20 countries and their top 10 cities
    Output: The attractiveness of each city in each time period (in the form of a 2-dimensional list), and a list composed of city names
    '''
    # Extracting city, country, and continent codes from all_geo
    all_geo = all_geo[['city', 'country_code', 'continent_code']]
    countries = list(set(all_geo['country_code']))
    all_geo.loc[:, 'country_id'] = all_geo['country_code'].apply(lambda x: countries.index(x))
    continents = list(set(all_geo['continent_code']))
    all_geo.loc[:, 'continent_id'] = all_geo['continent_code'].apply(lambda x: continents.index(x))
    cities = list(set(all_geo['city']))
    all_geo.loc[:, 'city_id'] = all_geo['city'].apply(lambda x: cities.index(x))
    all_geo = all_geo[['city_id', 'country_id', 'continent_id']]
    
    # Extracting and numbering the geographic information of all institutions in the parent container
    try:
        ins_geo = ins_geo[ins_geo['country_code'] == code]
    except:
        print("Please enter a valid country code")
        return

    countries = list(set(ins_geo['country_code']))
    ins_geo.loc[:, 'country_id'] = ins_geo['country_code'].apply(lambda x: countries.index(x))
    continents = list(set(ins_geo['continent_code']))
    ins_geo.loc[:, 'continent_id'] = ins_geo['continent_code'].apply(lambda x: continents.index(x))
    cities = list(set(ins_geo['city']))
    ins_geo.loc[:, 'city_id'] = ins_geo['city'].apply(lambda x: cities.index(x))
    ins_geo = ins_geo[['city_id', 'country_id', 'continent_id']]
    
    results = []
    for year in range(1960, 2011, 5):
        f_read = open('data/flow_matrices/flow_matrix[%d-%d].pkl' %(year, year+4), 'rb')
        this_mat = pickle.load(f_read)
        f_read.close()
        this_mat = this_mat.A
        results.append(estimate_by_slice(this_mat, 0, ins_geo, all_geo))
    f_read = open('data/flow_matrices/flow_matrix[%d-%d].pkl' %(2015, 2021), 'rb')
    this_mat = pickle.load(f_read)
    f_read.close()
    this_mat = this_mat.A
    results.append(estimate_by_slice(this_mat, 0, ins_geo, all_geo))
    return results, cities


In [63]:
def estimate_this_continent(code:str, all_geo, ins_geo):
    '''
    Input: Country code, geographic data of all institutions in the top 30 countries, geographic data of institutions in the top 20 countries and their top 10 cities
    Output: The attractiveness of each city in each time period (in the form of a 2-dimensional list), and a list composed of city names
    '''
    # Extracting city, country, and continent codes from all_geo
    all_geo = all_geo[['city', 'country_code', 'continent_code']]
    countries = list(set(all_geo['country_code']))
    all_geo.loc[:, 'country_id'] = all_geo['country_code'].apply(lambda x: countries.index(x))
    continents = list(set(all_geo['continent_code']))
    all_geo.loc[:, 'continent_id'] = all_geo['continent_code'].apply(lambda x: continents.index(x))
    cities = list(set(all_geo['city']))
    all_geo.loc[:, 'city_id'] = all_geo['city'].apply(lambda x: cities.index(x))
    all_geo = all_geo[['city_id', 'country_id', 'continent_id']]
    
    # Extracting and numbering the geographic information of all institutions in the parent container
    ins_geo = ins_geo[ins_geo['continent_code'] == code]

    countries = list(set(ins_geo['country_code']))
    ins_geo.loc[:, 'country_id'] = ins_geo['country_code'].apply(lambda x: countries.index(x))
    continents = list(set(ins_geo['continent_code']))
    ins_geo.loc[:, 'continent_id'] = ins_geo['continent_code'].apply(lambda x: continents.index(x))
    cities = list(set(ins_geo['city']))
    ins_geo.loc[:, 'city_id'] = ins_geo['city'].apply(lambda x: cities.index(x))
    ins_geo = ins_geo[['city_id', 'country_id', 'continent_id']]
    
    results = []
    for year in range(1960, 2011, 5):
        f_read = open('data/flow_matrices/flow_matrix[%d-%d].pkl' %(year, year+4), 'rb')
        this_mat = pickle.load(f_read).A
        f_read.close()
        results.append(estimate_by_slice(this_mat, 1, ins_geo, all_geo))
    f_read = open('data/flow_matrices/flow_matrix[%d-%d].pkl' %(2015, 2021), 'rb')
    this_mat = pickle.load(f_read).A
    f_read.close()
    results.append(estimate_by_slice(this_mat, 1, ins_geo, all_geo))
    return results, countries

In [64]:
def estimate_this_world(all_geo, ins_geo):
    '''
    Input: Country code, geographic data of all institutions in the top 30 countries, geographic data of institutions in the top 20 countries and their top 10 cities
    Output: The attractiveness of each city in each time period (in the form of a 2-dimensional list), and a list composed of city names
    '''
    # Extracting city, country, and continent codes from all_geo
    all_geo = all_geo[['city', 'country_code', 'continent_code']]
    countries = list(set(all_geo['country_code']))
    all_geo.loc[:, 'country_id'] = all_geo['country_code'].apply(lambda x: countries.index(x))
    continents = list(set(all_geo['continent_code']))
    all_geo.loc[:, 'continent_id'] = all_geo['continent_code'].apply(lambda x: continents.index(x))
    cities = list(set(all_geo['city']))
    all_geo.loc[:, 'city_id'] = all_geo['city'].apply(lambda x: cities.index(x))
    all_geo = all_geo[['city_id', 'country_id', 'continent_id']]
    
    # Extracting and numbering the geographic information of all institutions in the parent container
    # ins_geo = ins_geo[ins_geo['continent_code'] == code]

    countries = list(set(ins_geo['country_code']))
    ins_geo.loc[:, 'country_id'] = ins_geo['country_code'].apply(lambda x: countries.index(x))
    continents = list(set(ins_geo['continent_code']))
    ins_geo.loc[:, 'continent_id'] = ins_geo['continent_code'].apply(lambda x: continents.index(x))
    cities = list(set(ins_geo['city']))
    ins_geo.loc[:, 'city_id'] = ins_geo['city'].apply(lambda x: cities.index(x))
    ins_geo = ins_geo[['city_id', 'country_id', 'continent_id']]
    
    results = []
    for year in range(1960, 2011, 5):
        f_read = open('data/flow_matrices/flow_matrix[%d-%d].pkl' %(year, year+4), 'rb')
        this_mat = pickle.load(f_read).A
        f_read.close()
        results.append(estimate_by_slice(this_mat, 2, ins_geo, all_geo))
    f_read = open('data/flow_matrices/flow_matrix[%d-%d].pkl' %(2015, 2021), 'rb')
    this_mat = pickle.load(f_read).A
    f_read.close()
    results.append(estimate_by_slice(this_mat, 2, ins_geo, all_geo))
    return results, continents

In [65]:
all_geo = pd.read_hdf('data/top30ins_geo.h5') # Institutions with valid geographic information in the top 30 countries
ins_geo = pd.read_hdf('data/top20ins_geo.h5') # Institutions within the top 20 countries, each country's top 10 cities included

In [66]:
year_index = []
for year in range(1960, 2011, 5):
    year_index.append("%d-%d"%(year, year+4))
year_index.append("2015-2021")

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    writer = pd.ExcelWriter('data/attrativeness_countries.xlsx', engine='xlsxwriter')
    for continent in ins_geo['continent_code'].unique():
        results, countries = estimate_this_continent(continent, all_geo=all_geo, ins_geo=ins_geo)
        results_df = pd.DataFrame(results, columns=countries, index=year_index)
        results_df.to_excel(writer, sheet_name=continent)
    writer.close()

In [67]:
results, continents = estimate_this_world(all_geo, ins_geo)

/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_63898/1282359172.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_geo.loc[:, 'country_id'] = all_geo['country_code'].apply(lambda x: countries.index(x))
/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_63898/1282359172.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_geo.loc[:, 'continent_id'] = all_geo['continent_code'].apply(lambda x: continents.index(x))
/var/folders/5z/8hzvmhln5p36t4tdqc9c52z40000gn/T/ipykernel_63898/128

In [68]:
con_results_df = pd.DataFrame(results, columns=continents, index=year_index)

In [69]:
con_results_df

,EU,OC,SA,NA,AS
1960-1964,0.442022,0.104063,0.012884,0.311199,0.129832
1965-1969,0.435681,0.115962,0.011268,0.295305,0.141784
1970-1974,0.447247,0.116315,0.016469,0.278435,0.141534
1975-1979,0.449455,0.118189,0.015596,0.275648,0.141112
1980-1984,0.414103,0.095774,0.014366,0.307434,0.168323
1985-1989,0.417311,0.087607,0.017985,0.308346,0.168751
1990-1994,0.420211,0.087599,0.017786,0.299472,0.174933
1995-1999,0.406776,0.090315,0.025913,0.297507,0.179489
2000-2004,0.385625,0.091367,0.024152,0.289212,0.209644
2005-2009,0.359725,0.099116,0.025768,0.272009,0.243383


In [70]:
con_results_df.to_csv('data/attrativeness_continents.csv')

In [71]:
top20countries = ins_geo['country_code'].unique()

year_index = []
for year in range(1960, 2011, 5):
    year_index.append("%d-%d"%(year, year+4))
year_index.append("2015-2021")

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    writer = pd.ExcelWriter('data/attrativeness_cities.xlsx', engine='xlsxwriter')
    for country in top20countries:
        results, cities = estimate_this_country(country, all_geo, ins_geo)
        results_df = pd.DataFrame(results, columns=cities, index=year_index)
        results_df.to_excel(writer, sheet_name=country)
    writer.close()

In [72]:
def draw(results, list, country):
    %matplotlib inline


    plt.figure(figsize=(16,9))
    x = range(1960, 2016, 5)
    for i in range(10):
        plt.plot(x, results[:, i],'o-', label = list[i]) #每一列为一个城市的attractiveness变化数据
    plt.title(country)
    plt.xlabel("time")
    plt.ylabel("attractiveness")
    plt.legend(loc="best")
    plt.show()

In [73]:
# all_geo = pd.read_hdf('../SummerTeamData/filtered_ins_geo.h5') # 前30个国家中所有地理信息有效的机构
# ins_geo = pd.read_hdf('../SummerTeamData/filtered_ins_geo_v3.h5') # 前20个国家中每个国家的前10城市中的机构

# import warnings
# with warnings.catch_warnings():
#     warnings.simplefilter('ignore')
#     for country in ['US', 'CN', 'IN', 'JP', 'GB', 'FR', 'DE', 'BR', 'RU', 'ES', 'KR', 'CA',
#        'IT', 'AU', 'ID', 'IR', 'TR', 'TW', 'PL', 'MX']:
#         results, cities = estimate_this_country(country, all_geo, ins_geo)
#         results = np.array(results)
#         draw(results, cities, country)